<h1>Gradio Day!</h1>

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [1]:
#imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List 
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import ollama
from IPython.display import display, Markdown
import gradio as gr

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    openai = OpenAI(api_key=openai_api_key)
else:
    print("OpenAI API key not found.")

In [3]:
# connect to Open AI
openai = OpenAI()

In [4]:
# A generic system message - no more snarky adversarial AIs!
system_message = "You are a helpful assistant"

In [5]:
#Let's wrap a call to GPT-4o-mini in a simple function
def message_gpt(prompt):
    messages = [
        {"role":"system","content": system_message},
        {"role":"user","content":prompt}
    ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    return completion.choices[0].message.content

In [7]:
# This can reveal the "training cut off" or the most recent date in the training data
message_gpt("What is the training cut-off date for this model?")

'The training cut-off date for this model is October 2021.'

<H1> User Interface time!

In [12]:
# here is a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [9]:
shout("hello world")

Shout has been call with input hello world


'HELLO WORLD'

In [13]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never",js=force_dark_mode).launch(share=False)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input yo yo 


In [14]:
# Labelling inputs and outputs in Gradio 

view = gr.Interface(fn=shout,
                    inputs=[gr.Textbox(label="Your message",lines=6)],
                    outputs=[gr.Textbox(label="Response:",lines=8)],
                    flagging_mode="never")
view.launch(share=False)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input whatsup


In [15]:
# Calling the message_gpt method 

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs = [gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch(share=False)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [16]:
#Let's use Markdown to format our messages
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share=False)

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [17]:
# Let's create a call that streams back results

def stream_gpt(prompt):
    messages=[
        {"role":"system","content":system_message},
        {"role":"user","content":prompt}
    ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result+=chunk.choices[0].delta.content or ""
        yield result

In [18]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label = "Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share=False)

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [19]:
# A class represent a webpage
class Website:
    url:str
    title:str
    text:str


def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"    

In [21]:
system_message = "You are an asistant that analyzes the contents of a company website landing page \
    and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [22]:
def stream_brochure(company_name,url,model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model == "GPT":
        result=stream_gpt(prompt)
    else:
        raise ValueError("Unknown Model")
    yield from result

In [24]:
view = gr.Interface(
    fn=stream_brochure,
    inputs = [
        gr.Textbox(label="Company Name", placeholder="Enter the company name"),
        gr.Textbox(label="Landing page url including http or https"),
        gr.Dropdown(["GPT"],label="Select model")
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch(share=False)

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/Venkat_Tikkireddi/Desktop/Dev/GenAI_Udemy_EdDonner/genai-projects/venv/lib/python3.12/site-packages/gradio/queueing.py", line 716, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Venkat_Tikkireddi/Desktop/Dev/GenAI_Udemy_EdDonner/genai-projects/venv/lib/python3.12/site-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Venkat_Tikkireddi/Desktop/Dev/GenAI_Udemy_EdDonner/genai-projects/venv/lib/python3.12/site-packages/gradio/blocks.py", line 2240, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Venkat_Tikkireddi/Desktop/Dev/GenAI_Udemy_EdDonner/genai-projects/venv/lib/python3.12/site-packages/gradio/blocks.py", line 1759, in call_function
    prediction = await utils.asy